# ch03. 서울시 범죄 현황 데이터 분석
## 01-02강. 범죄 데이터 불러오기


In [1]:
from matplotlib import font_manager
f_path = "C:\WINDOWS\FONTS\malgun.ttf"
font_manager.FontProperties(fname=f_path).get_name()
from matplotlib import rc
rc("font", family='Malgun Gothic')
import matplotlib.pyplot as plt
# 마이너스로 한글이 깨지는 것을 방지하기 위함
plt.rcParams["axes.unicode_minus"] = False
%matplotlib inline
# 사용하는 모듈 불러오기
import pandas as pd
import numpy as np

In [5]:
# 데이터 불러오기
crime_raw_data = pd.read_csv("../data/02_crime/02. crime_in_Seoul.csv",
                             thousands=",", encoding="euc-kr")
crime_raw_data.head()
crime_raw_data.info()
crime_raw_data["죄종"].unique()
#죄종에서 nan 만 추출
crime_raw_data["죄종"].isnull()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65534 entries, 0 to 65533
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


0        False
1        False
2        False
3        False
4        False
         ...  
65529     True
65530     True
65531     True
65532     True
65533     True
Name: 죄종, Length: 65534, dtype: bool

In [8]:
#죄종에서 nan 만 추출
crime_raw_data[crime_raw_data["죄종"].isnull()]
# nan이 아닌 데이터만 추출해오기
crime_raw_data = crime_raw_data[crime_raw_data["죄종"].notnull()]
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 0 to 309
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 12.1+ KB


## 03- 04강 pandas pivot_table


In [13]:
# pivot_table 이론 연습
df = pd.read_excel("../data/02_crime/02. sales-funnel.xlsx")
df.head()
#name을 기준으로 정렬
pd.pivot_table(df, index=["Name"])
#index를 여러 개 지정할 수 있음
pd.pivot_table(df, index=["Name", "Rep", "Manager"])
pd.pivot_table(df, index=[ "Manager", "Rep"])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

In [27]:
# values또한 여러개를 지정할 수 있음
#aggfunc를 통해 함수를 사용 가능함
# 기본은 평균으로 되어 있음, 합산 등 다른 함수를 사용할 때는 aggfunc를 사용
pd.pivot_table(df, index=[ "Manager", "Rep"], values=["Price"],
                aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [16]:
# len도 적용 됨
pd.pivot_table(df, index=[ "Manager", "Rep"], values=["Price"],
                aggfunc= [np.mean, len])

mean   len
                                    Price Price
Manager       Rep                              
Debra Henley  Craig Booker   20000.000000     4
              Daniel Hilton  38333.333333     3
              John Smith     20000.000000     2
Fred Anderson Cedric Moss    27500.000000     4
              Wendy Yule     44250.000000     4

In [20]:
# 컬럼 분류를 지정 할 수 있다
pd.pivot_table(df, index=[ "Manager", "Rep"], 
               values=["Price"],
               columns=["Product"], 
                aggfunc=[np.sum])
#nan값에 대한 처리를 지정 fill_value
pd.pivot_table(df, index=[ "Manager", "Rep"], 
               values=["Price"],
               columns=["Product"], 
                aggfunc=[np.sum],
                fill_value=0)

sum                             
                              Price                             
Product                         CPU Maintenance Monitor Software
Manager       Rep                                               
Debra Henley  Craig Booker    65000        5000       0    10000
              Daniel Hilton  105000           0       0    10000
              John Smith      35000        5000       0        0
Fred Anderson Cedric Moss     95000        5000       0    10000
              Wendy Yule     165000        7000    5000        0

In [21]:
#aggfunc가 두개 이상으로 지정 가능하다. 
#margin = True를 사용하면 총계가 나온다
pd.pivot_table(df, index=[ "Manager", "Rep", "Product"], 
               values=["Price", "Quantity"],
                aggfunc=[np.sum, np.mean],
                fill_value=0,
                margins= True)


sum                   mean  \
                                          Price Quantity         Price   
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500.000000   
                            Maintenance    5000        2   5000.000000   
                            Software      10000        1  10000.000000   
              Daniel Hilton CPU          105000        4  52500.000000   
                            Software      10000        1  10000.000000   
              John Smith    CPU           35000        1  35000.000000   
                            Maintenance    5000        2   5000.000000   
Fred Anderson Cedric Moss   CPU           95000        3  47500.000000   
                            Maintenance    5000        1   5000.000000   
                            Software      10000        1  10000.000000   
              Wendy Yule    CPU          165000        7  82500.000000   
                            Maintenance    7000        3   7000.000000   
                            Monitor        5000        2   5000.000000   
All                                      522000       30  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

## 05-06강 서울시 범죄 현황 데이터 정리

In [29]:
crime_raw_data.head()
# 경찰서 이름을 인덱스로 정하고 np.sum을 이용
crime_station = crime_raw_data.pivot_table(
    crime_raw_data, index=["구분"], columns=["죄종", "발생검거"],
    aggfunc=[np.sum]
)
# pivot_table을 적용하면 column이나 index가 다중으로 잡힘
# column이 멀티로 잡히는 문제점이 발생하는 것을 확인
crime_station.head()
# 다중 컬럼에서 특정 컬럼 제거
crime_station.columns = crime_station.columns.droplevel([0, 1])
#다중 컬럼이 사라진 것을 확인
crime_station.columns
crime_station.head()
# 현재 인덱스는 경찰서 이름으로 되어 있음
# 경찰서 이름으로 지역구를 찾아내야함
crime_station.index

죄종       강간           강도          살인           절도              폭력        
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생
구분                                                                       
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0

---
## 구글 맵스 api 설정
- conda install -c conda-forge googlemaps
---
## 10- 11강 파이썬의 for 문
- 판다스를 사용 할 때 for 문을 사용하면 가독률이 떨어짐
- 판다스 데이터 프레임으로 반복문을 만들 때 iterrows()라는 옵션을 사용하면 편함
- 받을 때 인덱스와 내용으로 나누어 받는 것을 주의


In [38]:
# 간단한 for 문 예제
for n in [1,2,3,4]:
    print(n)
# 조금 복잡한 for 문 예제
for n in range(0, 10):
    print(n**2)
## list comprehension을 이용해서 코드를 한줄로
[n ** 2 for n in range(0,10)]


1
2
3
4
0
1
4
9
16
25
36
49
64
81


---
## 12 - 강. 구글 맵스에서 구별 정보를 얻어서 데이터를 정리하기

In [36]:
# 구글맵 import
import googlemaps

In [34]:
# api키
#개인정보를 위해 삭제됨
gmaps_key = "개인정보를 위해 삭제됨"
gmaps = googlemaps.Client(key=gmaps_key)
gmaps.geocode("서울영등포경찰서", language="ko")
print(tmp[0].get("geometry")["location"]["lat"])
print(tmp[0].get("geometry")["location"]["lng"])
print(tmp[0].get("formatted_address"))
tmp.split()
tmp.split()[2]

In [ ]:
# 경찰서 전체에 대해 google map을 이용해서 주소와 위치정보 얻기
count = 0 # 작업이 잘 되고 있는지 확인하기 위한 변수
for idx, rows in crime_station.iterrows():
    # 구글 검색을 용이하게 하기위해 검색어를 상세하게 잡아줌
    
    station_name = "서울" + str(idx)+ "경찰서"
    tmp = gmaps.geocode(station_name, language="ko")
    # 구 이름 잡는 과정
    tmp[0].get("formatted_address")
    tmp_gu = tmp[0].get("formatted_address")
    
    lat = tmp[0].get("geometry")["location"]["lat"]
    lng = tmp[0].get("geometry")["location"]["lng"]
    # loc 기능을 사용해 행(idx)과 열(lat, lng, 구별)을 지정해 
    # 검색에서 얻은 정보를 기록함
    crime_station.loc[idx, "lat"] = lat
    crime_station.loc[idx, "lng"] = lat
    crime_station.loc[idx, "구별"] = tmp_gu.split()[2]
    print(count)
    count += 1
# 컬럼이 두줄로 되어 있는 것을 확인 각 죄종당 발생 검거를 합쳐서 한줄로 만들기    
crime_station.head()

In [ ]:
tmp = [
    crime_station.columns.get_level_values(0)[n] +
    crime_station.columns.get_level_values(1)[n]
    for n in range(0, len(crime_station.columns.get_level_values(0)))
]

crime_station.columns = tmp
crime_station.head()
# 데이터 저장
crime_station.to_csv("../data/20220223_ crime_in_Seoul_raw.csv", sep=",", encoding="utf-8")

## 14- 15강. 구별로 데이터 변경하기

In [2]:
# 데이터 불러오기
crime_anal_station = pd.read_csv(
    "../data/02. crime_in_Seoul_raw.csv", index_col= 0, encoding= "utf-8"
)
crime_anal_station.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,구별,lat,lng
구분,,,,,,,,,,,,,
강남,269.0,339.0,26.0,24.0,3.0,3.0,1129.0,2438.0,2096.0,2336.0,강남구,37.509435,127.066958
강동,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0,강동구,37.528511,127.126822
강북,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0,강북구,37.637388,127.027324
강서,239.0,275.0,10.0,10.0,10.0,9.0,1070.0,1952.0,2768.0,3204.0,양천구,37.539783,126.829997
관악,264.0,322.0,10.0,12.0,7.0,6.0,937.0,2103.0,2707.0,3235.0,관악구,37.474379,126.950975


In [4]:
# pivot_table을 이용해서 구별로 정리
# aggfunc를 이용해서 sum을 사용
crime_anal_gu = pd.pivot_table(crime_anal_station, index="구별", aggfunc= np.sum)
del crime_anal_gu["lat"]
del crime_anal_gu["lng"]
crime_anal_gu.head()

#검거율 만들기
crime_anal_gu["강도검거"] / crime_anal_gu["강도발생"]
# 하나의 컬럼을 다른 컬럼으로 나누기
crime_anal_gu[["강도검거", "살인검거"]].div(crime_anal_gu["강도검거"], axis=0)

,강도검거,살인검거
구별,,
강남구,1.0,0.119048
강동구,1.0,0.384615
강북구,1.0,1.500000
관악구,1.0,0.764706
광진구,1.0,0.666667
구로구,1.0,0.692308
노원구,1.0,0.666667
도봉구,1.0,0.600000
동대문구,1.0,0.416667


In [5]:
# 다수의 컬럼을 다른 컬럼으로 나누기
num = ["강간검거", "강도검거", "살인검거", "절도검거", "폭력검거"]
den = ["강간발생", "강도발생", "살인발생", "절도발생", "폭력발생"]
crime_anal_gu[num].div(crime_anal_gu[den].values)

# 다수의 컬럼을 다수의 컬럼으로 나누기
target = ["강간검거율", "강도검거율", "살인검거율", "절도검거율", "폭력검거율"]
num = ["강간검거", "강도검거", "살인검거", "절도검거", "폭력검거"]
den = ["강간발생", "강도발생", "살인발생", "절도발생", "폭력발생"]

crime_anal_gu[target] = crime_anal_gu[num].div(crime_anal_gu[den].values) * 100
crime_anal_gu.head()


,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,,,,,,
강남구,413.0,516.0,42.0,39.0,5.0,5.0,1918.0,3587.0,3527.0,4002.0,80.038760,107.692308,100.000000,53.470867,88.130935
강동구,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0,95.000000,92.857143,125.000000,51.425314,86.996047
강북구,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
관악구,407.0,497.0,17.0,19.0,13.0,12.0,1591.0,3367.0,4653.0,5428.0,81.891348,89.473684,108.333333,47.252747,85.722181
광진구,234.0,279.0,6.0,11.0,4.0,4.0,1057.0,2636.0,2011.0,2392.0,83.870968,54.545455,100.000000,40.098634,84.071906


In [6]:
# 필요없는 검거쪽 컬럼 삭제
del crime_anal_gu["강간검거"]
del crime_anal_gu["강도검거"]
del crime_anal_gu["살인검거"]
del crime_anal_gu["절도검거"]
del crime_anal_gu["폭력검거"]

crime_anal_gu.head()

,강간발생,강도발생,살인발생,절도발생,폭력발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,516.0,39.0,5.0,3587.0,4002.0,80.038760,107.692308,100.000000,53.470867,88.130935
강동구,160.0,14.0,4.0,1754.0,2530.0,95.000000,92.857143,125.000000,51.425314,86.996047
강북구,217.0,5.0,7.0,1222.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
관악구,497.0,19.0,12.0,3367.0,5428.0,81.891348,89.473684,108.333333,47.252747,85.722181
광진구,279.0,11.0,4.0,2636.0,2392.0,83.870968,54.545455,100.000000,40.098634,84.071906


In [7]:
# 단순 계산으로 검거율이 100이 넘는 경우도 있음
# 100 이상의 수치를 100으로 고정
crime_anal_gu[crime_anal_gu[target] > 100] = 100
crime_anal_gu.head()

# 다른 방법
target = ["강간검거율", "강도검거율", "살인검거율", "절도검거율", "폭력검거율"]
for column in con_list:
    crime_anal_gu.loc[crime_anal_gu[target] > 100, column] = 100

,강간발생,강도발생,살인발생,절도발생,폭력발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,516.0,39.0,5.0,3587.0,4002.0,80.038760,100.000000,100.000000,53.470867,88.130935
강동구,160.0,14.0,4.0,1754.0,2530.0,95.000000,92.857143,100.000000,51.425314,86.996047
강북구,217.0,5.0,7.0,1222.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
관악구,497.0,19.0,12.0,3367.0,5428.0,81.891348,89.473684,100.000000,47.252747,85.722181
광진구,279.0,11.0,4.0,2636.0,2392.0,83.870968,54.545455,100.000000,40.098634,84.071906


In [8]:
# 컬럼명 간단하게 변경
crime_anal_gu.rename(
    columns= {"강간발생" : "강간", "강도발생" : "강도", "살인발생" : "살인", 
              "절도발생" : "절도", "폭력발생" : "폭력"}, inplace= True
)
crime_anal_gu.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,516.0,39.0,5.0,3587.0,4002.0,80.038760,100.000000,100.000000,53.470867,88.130935
강동구,160.0,14.0,4.0,1754.0,2530.0,95.000000,92.857143,100.000000,51.425314,86.996047
강북구,217.0,5.0,7.0,1222.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
관악구,497.0,19.0,12.0,3367.0,5428.0,81.891348,89.473684,100.000000,47.252747,85.722181
광진구,279.0,11.0,4.0,2636.0,2392.0,83.870968,54.545455,100.000000,40.098634,84.071906


## 16- 17강. 범죄 현황 데이터 최종 정리


In [10]:
# 구별 데이터에서 발생건수 정규화 데이터를 생성
# 정규화해서 최곳값은 1, 최솟값은 0
col = ["살인", "강도", "강간", "절도", "폭력"]
crime_anal_gu_norm = crime_anal_gu[col] / crime_anal_gu[col].max()
crime_anal_gu_norm.head()

#기존에 있던 검거율 데이터 추가
col02 = ["살인검거율", "강도검거율", "강간검거율", "절도검거율", "폭력검거율"]
crime_anal_gu_norm[col02] = crime_anal_gu[col02]
crime_anal_gu_norm.head()

,살인,강도,강간,절도,폭력,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,0.357143,1.000000,1.000000,0.977118,0.733773,100.000000,100.000000,80.038760,53.470867,88.130935
강동구,0.285714,0.358974,0.310078,0.477799,0.463880,100.000000,92.857143,95.000000,51.425314,86.996047
강북구,0.500000,0.128205,0.420543,0.332879,0.509351,85.714286,80.000000,73.271889,54.991817,89.344852
관악구,0.857143,0.487179,0.963178,0.917189,0.995233,100.000000,89.473684,81.891348,47.252747,85.722181
광진구,0.285714,0.282051,0.540698,0.718060,0.438577,100.000000,54.545455,83.870968,40.098634,84.071906


In [11]:
# 인구와 cctv 데이터 추가
# 먼저 불러옴
result_CCTV = pd.read_csv(
    "../data/20220222_cctv_result.csv", encoding="UTF-8", index_col="구별"
)
crime_anal_gu_norm[["인구수", "CCTV"]] = result_CCTV[["인구수", "소계"]]
crime_anal_gu_norm.head()

,살인,강도,강간,절도,폭력,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율,인구수,CCTV
구별,,,,,,,,,,,,
강남구,0.357143,1.000000,1.000000,0.977118,0.733773,100.000000,100.000000,80.038760,53.470867,88.130935,561052,3238
강동구,0.285714,0.358974,0.310078,0.477799,0.463880,100.000000,92.857143,95.000000,51.425314,86.996047,440359,1010
강북구,0.500000,0.128205,0.420543,0.332879,0.509351,85.714286,80.000000,73.271889,54.991817,89.344852,328002,831
관악구,0.857143,0.487179,0.963178,0.917189,0.995233,100.000000,89.473684,81.891348,47.252747,85.722181,520929,2109
광진구,0.285714,0.282051,0.540698,0.718060,0.438577,100.000000,54.545455,83.870968,40.098634,84.071906,372298,878


In [12]:
# 정규화된 범죄 발생 건수 전체의 평균을 구해서 범죄의 대푯값으로 사용
# 2차원 이상의 행렬로 되어 있음
# 판다스에선 axis = 1 행을 기준으로 계산 axis = 0 은 열을 기준으로 계산
col = ["살인", "강도", "강간", "절도", "폭력"]
crime_anal_gu_norm["범죄"] = np.mean(crime_anal_gu_norm[col], axis=1)
crime_anal_gu_norm.head()

# 검거율의 평균을 구해서 검거의 대푯값으로 사용
col02 = ["살인검거율", "강도검거율", "강간검거율", "절도검거율", "폭력검거율"]
crime_anal_gu_norm["검거"] = np.mean(crime_anal_gu_norm[col02], axis=1)
crime_anal_gu_norm.head()


,살인,강도,강간,절도,폭력,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율,인구수,CCTV,범죄,검거
구별,,,,,,,,,,,,,,
강남구,0.357143,1.000000,1.000000,0.977118,0.733773,100.000000,100.000000,80.038760,53.470867,88.130935,561052,3238,0.813607,84.328112
강동구,0.285714,0.358974,0.310078,0.477799,0.463880,100.000000,92.857143,95.000000,51.425314,86.996047,440359,1010,0.379289,85.255701
강북구,0.500000,0.128205,0.420543,0.332879,0.509351,85.714286,80.000000,73.271889,54.991817,89.344852,328002,831,0.378196,76.664569
관악구,0.857143,0.487179,0.963178,0.917189,0.995233,100.000000,89.473684,81.891348,47.252747,85.722181,520929,2109,0.843984,80.867992
광진구,0.285714,0.282051,0.540698,0.718060,0.438577,100.000000,54.545455,83.870968,40.098634,84.071906,372298,878,0.453020,72.517393
